# Adiciona número de turistas à Matriz de Origem e Destino

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Utilizando o engine 'openpyxl' para ler o arquivo Excel
categoria_turismo = pd.read_excel(
    'data/excel/xlsx/categoria_turismo.xlsx', 
    skiprows=4,  # Pular as quatro primeiras linhas
    header=None,  # Não usar o cabeçalho da quinta linha original
    engine='openpyxl'  # Usar o openpyxl para leitura
)

# Definir o cabeçalho manualmente com os itens fornecidos
categoria_turismo.columns = [
    'uf', 'municipio', 'regiao_turistica', 'visitantes_internacionais', 
    'visitantes_domesticos', 'hospedagem', 'empregos', 'impostos_federais', 'categoria'
]

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
categoria_turismo.head()

,uf,municipio,regiao_turistica,visitantes_internacionais,visitantes_domesticos,hospedagem,empregos,impostos_federais,categoria
0,AC,Assis Brasil,Caminhos do Pacífico,0,27381,1,1,0,D
1,AC,Cruzeiro do Sul,Caminhos das Aldeias e da Biodiversidade,481,24475,11,116,241310,C
2,AC,Epitaciolândia,Caminhos do Pacífico,0,4785,6,31,0,D
3,AC,Rio Branco,Caminhos da Revolução,4602,464108,30,376,2845034,A
4,AC,Rodrigues Alves,Caminhos das Aldeias e da Biodiversidade,0,0,0,0,0,E


In [4]:
municipios = gpd.read_file("data/shapefile/centroide/municipios_centroide.shp")

In [5]:
municipios

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,1100015,Alta Floresta D'Oeste,RO,7067.127,POINT (-62.27475 -12.47091)
1,1100023,Ariquemes,RO,4426.571,POINT (-62.95714 -9.95196)
2,1100031,Cabixi,RO,1314.352,POINT (-60.63988 -13.47493)
3,1100049,Cacoal,RO,3793.000,POINT (-61.3248 -11.30138)
4,1100056,Cerejeiras,RO,2783.300,POINT (-61.26104 -13.20356)
...,...,...,...,...,...
5567,5222005,Vianópolis,GO,954.115,POINT (-48.44085 -16.81296)
5568,5222054,Vicentinópolis,GO,733.794,POINT (-49.87271 -17.72376)
5569,5222203,Vila Boa,GO,1052.593,POINT (-47.06238 -14.9933)
5570,5222302,Vila Propício,GO,2181.593,POINT (-48.8136 -15.26987)


In [6]:
def remover_duplicatas(categoria_turismo, municipios):
    # Fazer a junção entre categoria_turismo e municipios com base em 'municipio' e 'uf', além de 'NM_MUN' e 'SIGLA_UF'
    categoria_cod_merged = categoria_turismo.merge(
        municipios, 
        how='left', 
        left_on=['municipio', 'uf'], 
        right_on=['NM_MUN', 'SIGLA_UF']
    )
    
    # Contar duplicatas com base nas colunas 'municipio', 'uf', 'NM_MUN', e 'SIGLA_UF'
    duplicatas = categoria_cod_merged.groupby(['municipio', 'uf', 'NM_MUN', 'SIGLA_UF']).size().reset_index(name='counts')

    # Filtrar os casos em que há mais de uma correspondência
    duplicatas = duplicatas[duplicatas['counts'] > 1]

    # Identificar os índices das duplicatas
    duplicatas_indices = categoria_cod_merged[
        categoria_cod_merged.set_index(['municipio', 'uf']).index.isin(duplicatas.set_index(['municipio', 'uf']).index)
    ].index

    # Remover as duplicatas da base categoria_turismo original
    categoria_turismo_sem_duplicatas = categoria_turismo.drop(duplicatas_indices)

    return categoria_turismo_sem_duplicatas

In [7]:
cat_turismo_filtrado = remover_duplicatas(categoria_turismo, municipios)

In [8]:
cat_turismo_filtrado.columns

Index(['uf', 'municipio', 'regiao_turistica', 'visitantes_internacionais',
       'visitantes_domesticos', 'hospedagem', 'empregos', 'impostos_federais',
       'categoria'],
      dtype='object')

In [9]:
# Fazer o merge para adicionar a coluna 'cod_mun' à base cat_turismo_filtrado
cat_turismo_filtrado = cat_turismo_filtrado.merge(
    municipios[['NM_MUN', 'CD_MUN', 'SIGLA_UF']], 
    how='left', 
    left_on=['municipio', 'uf'], 
    right_on=['NM_MUN', 'SIGLA_UF']
)

# Remover as colunas de junção 'NM_MUN' e 'SIGLA_UF'
cat_turismo_filtrado.drop(columns=['NM_MUN', 'SIGLA_UF'], inplace=True)

# Renomear a coluna 'CD_MUN' para 'cod_mun'
cat_turismo_filtrado.rename(columns={'CD_MUN': 'cod_mun'}, inplace=True)

In [10]:
cat_turismo_filtrado.columns

Index(['uf', 'municipio', 'regiao_turistica', 'visitantes_internacionais',
       'visitantes_domesticos', 'hospedagem', 'empregos', 'impostos_federais',
       'categoria', 'cod_mun'],
      dtype='object')

In [11]:
matriz_od_info_mun= pd.read_csv('data/csv/matriz_od_info_mun.csv', delimiter = ',')

In [12]:
matriz_od_info_mun.columns

Index(['mun_origem', 'mun_destino', 'distancia', 'pib_origem', 'pib_destino',
       'pop_origem', 'pop_destino'],
      dtype='object')

In [13]:
def criar_matriz_od_turismo(matriz_od_info_mun, cat_turismo_filtrado):
    """
    Função para criar a base 'matriz_od_turismo' adicionando as colunas 'turistas_origem' e 'turistas_destino'
    à base 'matriz_od_info_mun'. A soma de 'visitantes_internacionais' e 'visitantes_domesticos' de 'cat_turismo_filtrado'
    será associada a 'mun_origem' e 'mun_destino' via 'cod_mun' da própria base 'cat_turismo_filtrado'.
    
    Parâmetros:
    matriz_od_info_mun (DataFrame): Base original com as informações de origem e destino dos municípios.
    cat_turismo_filtrado (DataFrame): Base de dados com informações turísticas por município, incluindo 'cod_mun'.
    
    Retorna:
    DataFrame: 'matriz_od_turismo' com as novas colunas 'turistas_origem' e 'turistas_destino'.
    """
    
    # Converter as colunas 'mun_origem' e 'mun_destino' para strings para compatibilidade com 'cod_mun'
    matriz_od_info_mun['mun_origem'] = matriz_od_info_mun['mun_origem'].astype(str)
    matriz_od_info_mun['mun_destino'] = matriz_od_info_mun['mun_destino'].astype(str)
    cat_turismo_filtrado['cod_mun'] = cat_turismo_filtrado['cod_mun'].astype(str)
    
    # Criar a coluna 'turistas' em cat_turismo_filtrado somando 'visitantes_internacionais' e 'visitantes_domesticos'
    cat_turismo_filtrado['turistas'] = cat_turismo_filtrado['visitantes_internacionais'] + cat_turismo_filtrado['visitantes_domesticos']
    
    # Fazer a junção para adicionar 'turistas_origem' baseado em 'mun_origem'
    matriz_od_turismo = matriz_od_info_mun.merge(
        cat_turismo_filtrado[['cod_mun', 'turistas']], 
        how='left', 
        left_on='mun_origem', 
        right_on='cod_mun'
    ).rename(columns={'turistas': 'turistas_origem'})
    
    # Fazer a junção para adicionar 'turistas_destino' baseado em 'mun_destino'
    matriz_od_turismo = matriz_od_turismo.merge(
        cat_turismo_filtrado[['cod_mun', 'turistas']], 
        how='left', 
        left_on='mun_destino', 
        right_on='cod_mun'
    ).rename(columns={'turistas': 'turistas_destino'})
    
    # Remover as colunas auxiliares 'cod_mun_x' e 'cod_mun_y' apenas se existirem
    colunas_a_remover = ['cod_mun_x', 'cod_mun_y']
    matriz_od_turismo.drop(columns=[col for col in colunas_a_remover if col in matriz_od_turismo.columns], inplace=True)
    
    return matriz_od_turismo

In [14]:
matriz_od_turismo = criar_matriz_od_turismo(matriz_od_info_mun, cat_turismo_filtrado)

In [15]:
# Salvando a matriz_od_turismo como um arquivo CSV
output_path_csv = 'data/csv/matriz_od_turismo.csv'
matriz_od_turismo.to_csv(output_path_csv, index=False)